# STM Transit Delay Data Modeling

## Overview

This notebook explores tree-based regression and classification models in order to find the one that predicts the delay with the best accuracy.

## Data Description

`trip_id`: Unique identifier for the transit trip.<br>
`route_id`: Unique identifier for a bus or metro line.<br>
`stop_id`: Unique identifier of a stop.<br>
`stop_lat`, `stop_lon`: Stop coordinates.<br>
`stop_distance`: Distance between the previous and current stop, in meters.<br>
`stop_sequence`: Sequence of the stop, for ordering.<br>
`trip_progress`: How far along the trip is the vehicle, from 0 (first stop) to 1 (last stop).<br>
`stop_has_alert`: Indicates if there's a message about the stop being moved or cancelled.<br>
`sch_rel_Scheduled`, `sch_rel_Skipped`: States of the schedule.<br>
`wheelchair_boarding`: Indicates if the stop is accessible for people in wheelchair<br>
`month`, `day_of_week`, `hour`: Month, day of week and hour of the scheduled arrival time.<br>
`is_weekend`: Indicates if the day of the arrival time is on the weekend.<br>
`is_peak_hour`: Indicates of the day and hour of the arrival time is at peak hour (weekdays between 7-9am and 4-6pm).<br>
`incident_count`: Number of incidents within 500 meters of the stop.<br>
`incident_nearby`: Indicates if an incident happened within 500 meters of the stop.<br>
`temperature`: Air temperature at 2 meters above ground, in Celsius.<br>
`precipitation`: Total precipitation (rain, showers, snow) sum of the preceding hour, in millimeters.<br>
`windspeed`: Wind speed at 10 meters above ground, in km/h.<br>
`weather_Cloudy`, `weather_Drizzle`, `weather_Rain`: Indicates the weather category.<br>
`delay`: Difference between real and scheduled arrival time, in seconds.<br>
`delay_class`: Delay category, from 0 (early) to 4 (late).<br>

## Imports

In [1]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import shap
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import plot_importance, XGBClassifier, XGBRegressor

In [2]:
df = pd.read_parquet('../data/preprocessed.parquet')
print(f'Number of rows: {len(df)}')

Number of rows: 1557996


## Split the data

In [3]:
# Train/Test Split (80-20) (stratified by delay_class)
df_train, df_temp = train_test_split(
  df,
  test_size=0.2,
  stratify=df['delay_class'],
  random_state=42
)

In [4]:
# Sample for faster experimentation (25% of the full data)
df_train_sample, _ = train_test_split(
  df_train,
  train_size=0.25,
  stratify=df_train['delay_class'],
  random_state=42
)

In [ ]:
# Check if class distribution is preserved
print(df['delay_class'].value_counts(normalize=True))
print(df_train_sample['delay_class'].value_counts(normalize=True))

delay_class
1    0.860496
2    0.123251
0    0.016254
Name: proportion, dtype: float64
delay_class
1    0.860497
2    0.123251
0    0.016252
Name: proportion, dtype: float64


In [ ]:
# Separate features and target
feature_cols = [col for col in df.columns if col not in ['delay', 'delay_log', 'delay_class']]

In [ ]:
# Sample train set
X_train_sample = df_train_sample[feature_cols]
y_reg_train_sample = df_train_sample['delay_log']
y_class_train_sample = df_train_sample['delay_class']

In [ ]:
# Full train set
X_train = df_train[feature_cols]
y_reg_train = df_train['delay_log']
y_class_train= df_train['delay_class']

In [ ]:
# Validation and test set
df_val, df_test = train_test_split(
	df_temp,
  	test_size=0.5,
    stratify=df_temp['delay_class'],
  	random_state=42
)

del df_temp

X_val = df_val[feature_cols]
y_reg_val = df_val['delay_log']
y_class_val = df_val['delay_class']

X_test = df_test[feature_cols]
y_reg_test = df_test['delay_log']
y_class_test = df_test['delay_class']

Since only tree-based models are explored in this project, **scaling is not needed** because the models are not sensitive to the absolute scale or distribution of the features.

## Regression Model

### Random Forest Regressor

#### Fit Model

In [10]:
# Fit base model
rf_reg_base = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_reg_base.fit(X_train_sample, y_reg_train_sample)

RandomForestRegressor(n_jobs=-1, random_state=42)

#### Evaluate Model

In [11]:
# Create dataframe to track metrics
metrics_df = pd.DataFrame(columns=['model', 'params', 'MAE', 'RMSE', 'R²'])

In [12]:
def add_metrics(metrics_df:pd.DataFrame, model, y_pred:pd.Series, y_val:pd.Series, model_name:str) -> pd.DataFrame:
	mae = mean_absolute_error(y_val, y_pred)
	rmse = root_mean_squared_error(y_val, y_pred)
	r2 = r2_score(y_val, y_pred)

	metrics_df.loc[len(metrics_df)] = [model_name, model.get_params(), mae, rmse, r2]
	return metrics_df

In [13]:
# Calculate metrics
y_pred = rf_reg_base.predict(X_val)
metrics_df = add_metrics(metrics_df, rf_reg_base, y_pred, y_reg_val, 'rf_reg_base')
metrics_df[['model', 'MAE', 'RMSE', 'R²']]

,model,MAE,RMSE,R²
0,rf_reg_base,107.140325,201.449965,-0.069803


**MAE**<br>
On average, the predictions are off by 73 seconds, which is not very good.

**RMSE**<br>
The higher RMSE compared to MAE suggests that there are some significant prediction errors that influence the overall error metric.

**R²**<br>
The model explains 20% of the variance in the actual delays, which is not good, but understandable considering how noisy and random transit delays can be (weather, mechanical issues, accidents, etc.)

In [ ]:
def plot_residuals(y_pred: pd.Series, y_true:pd.Series, model_name:str) -> None:
	fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 7))

	# Predicted vs. actual values
	ax1.scatter(x=y_pred, y=y_true)
	ax1.set_title('Predicted vs. Actual values')
	ax1.set_xlabel('Predicted delay (seconds)')
	ax1.set_ylabel('Actual delay (seconds)')
	ax1.grid(True)

	# Residuals
	residuals = y_true - y_pred
	ax2.scatter(x=y_pred, y=residuals)
	ax2.set_title('Residual Plot')
	ax2.set_xlabel('Predicted Delay (seconds)')
	ax2.set_ylabel('Residuals (seconds)')
	ax2.axhline(0, linestyle='--', color='orange')
	ax2.grid(True)

	fig.suptitle('Residual Analysis', fontsize=18)
	fig.tight_layout()
	fig.savefig(f'../images/residual_analysis_{model_name}.png', bbox_inches='tight')
	plt.show()

In [ ]:
# Plot residual analysis
plot_residuals(y_pred, y_reg_val, 'rf_reg_base')

The predicted vs. actual plot shows a scatter of points, with some spread around the diagonal line, which indicates the model's predictions are not perfect but are reasonable. The spread of points suggests the model has a reasonable fit but doesn't handle extreme delays very well.

From the residual plot, it seems that the residuals are fairly spread out, with visible funnel shapes and most of the data clustered around 0, but also a significant spread both above and below the 0 line. This suggests there is no clear pattern in residuals (not biased in one direction). Also, it suggests the residuals appear to increase as the predicted delay increases, meaning the model prediction errors are larger for higher predicted delays (heteroscedasticity). This means the model is better at predicting small delays than large delays.

#### Hyperparameter tuning

Due to the large volume of data, the Randomized Search will be run instead of a Grid Search.

In [ ]:
# Randomized search (15 combinations) with 3-fold CV
param_dist = {
    'n_estimators': list(range(100, 700, 100)),
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2', 0.8],
    'bootstrap': [True, False],
    'max_leaf_nodes': [None, 50, 100]
}

scoring = {
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
    'RMSE': make_scorer(root_mean_squared_error, greater_is_better=False),
    'R2': make_scorer(r2_score)
}

random_search_rf = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    verbose=2,
    scoring=scoring,
    refit='RMSE',
    n_jobs=-1,
    random_state=42
)

random_search_rf.fit(X_train_sample, y_reg_train_sample)

In [ ]:
# Best model
rf_reg_tuned = random_search_rf.best_estimator_
best_params = random_search_rf.best_params_

In [ ]:
# Calculate metrics
y_pred = rf_reg_tuned.predict(X_val)
metrics_df = add_metrics(metrics_df, rf_reg_tuned, y_pred, y_reg_val, 'rf_reg_tuned')
metrics_df[['model', 'MAE', 'RMSE', 'R²']]

There is a slight improvement from the base model to the tune model.

#### Feature Importances

In [ ]:
importances = rf_reg_tuned.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_reg_tuned.estimators_], axis=0)

In [ ]:
# Sort predictors
feature_importances = pd.Series(importances, index=X_train_sample.columns)
feature_importances = feature_importances.sort_values(ascending=False)
feature_importances

In [ ]:
# Plot the impurity-based importance (MDI)
fig, ax = plt.subplots()
feature_importances.plot.bar(yerr=std, ax=ax)
ax.set_title('Feature importances using MDI')
ax.set_ylabel('Mean decrease in impurity')
fig.tight_layout()
plt.savefig('../images/mdi_rf_reg_tuned.png', bbox_inches='tight')
plt.show()

**Top Features**

- `stop_distance`: Most important feature. Suggests delays are highly correlated with the distance between stops. Long gaps might lead to longer travel times or greater variability.
- `stop_lat` and `stop_lon`: Spatial location is very predictive, maybe due to geography (e.g., congestion in downtown vs. suburbs).
- `vehicle_bearing`: Direction of the bus (e.g. against the traffic) might lead to a bus being ahead or behind schedule.
- `trip_phase_end`: Delays likely accumulate or reduce toward trip end.
- `pressure`, `temperature`: These weather details add some value, may affect passenger behavior

**Moderately Important Features**
- `wind_direction`, `windspeed`, `relative_humidity`, `cloud-cover`: These weather details add modest value, it may affect vehicle's performance.
- `vehicle_speed`: Vehicle dynamics are important, it might indicate if a bus is ahead or behind schedule.
- `is_peak_hour`, `day_of_week`: Vehicles are most likely to be late on certain days and rush hour.
- `occupancy_status`: A vehicle with more passengers is more likely to be late.
- `stop_has_alert`: A stop being moved or cancelled can definitely affect delays.
- `route_type_Day`: A day route is more prone to delays than a night route.

**Least Important Features**
- `precipitation`: Rain precipitation doesn't really affect delays. The data was collected in spring. Snow precicipation would have probably have got have more impact.
- `vehicle_in_transit`, `wheelchair_boarding`, `route_type_Night`, `sch_rel_Scheduled`: These trip or vehicle related features have little impact.
- `time_of_day_night`, `time_of_day_night`, `time_of_day_midday`, `is_weekend`: Those time-related features didn't affect the delay.
- `incident_count`, `incident_nearby`: Incidents have near-zero impact, probably because of the sparse data.

#### SHAP Summary Plot

In [ ]:
# SHAP can be slow with large datasets
X_val_sample = X_val[:100]

# Build explainer
explainer = shap.TreeExplainer(rf_reg_tuned, feature_perturbation='tree_path_dependent')
shap_values = explainer.shap_values(X_val_sample)

In [ ]:
# Summary plot (bar chart)
shap.summary_plot(shap_values, X_val_sample, plot_type='bar', show=False)

plt.title('SHAP Bar Plot (Global Impact)', fontsize=18)
plt.tight_layout()
plt.savefig('../images/shap_barplot_rf_reg_tuned.png', bbox_inches='tight')
plt.show()

**Global Impact**

- `trip_phase_end`: Most impactful feature. End-of-trip status strongly influences delay, likely due to delay accumulation over time.
- `is_peak_hour`: Vehicles are most likely to be late at rush hour.
- `stop_lon`, `stop_lat`: Geography is highly predictive, possibly related to location-specific traffic patterns (e.g., downtown vs. suburbs).
- `stop_distance`: Longer gaps between stops relate to more variable delays.
- `trip_phase_middle`: A vehicle being half way through the trip is likely to accumulate delays.
- `pressure`, `temperature`, `relative_humidity`, `windspeed`, `wind_direction`, `cloud_cover`: Weather conditions can have an impact on the vehicle performance and passenger behaviour.
- `time_of_day_morning`: Morning peaks influence delays significantly.
- `vehicle_bearing`, `vehicle_speed`: Vehicle dynamics are important. Speed and direction might indicate if a bus is ahead or behind schedule.
- `stop_has_alert`, `day_of_week`, `vehicle_in_transit`, `route_type_Day`: Contrinute minimally on average.


In [ ]:
# Summary plot (beeswarm: shows direction and spread)
shap.summary_plot(shap_values, X_val_sample, show=False)

plt.title('SHAP Beeswarm Plot (Local + Global Insight)', fontsize=18)
plt.tight_layout()
plt.savefig('../images/shap_beeswarm_plot_rf_reg_tuned.png', bbox_inches='tight')
plt.show()

**Local + Global Insight**

- `trip_phase_end`: High values (red) → large positive SHAP → End-of-trip increases predicted delay.
- `is_peak_hour`: Being true (red) increases predicted delay, which is expected.
- `stop_lon`, `stop_lat`: Certain coordinated (red or blue) drive delay up or down, likely tied to location-specific bottlenecks.
- `stop_distance`: High distances (red) contribute positively to delay.
- `trip_phase_middle`: Similarly to trip_phase_end, being half way through increases predicted delay.
- Weather features (`pressure`, `wind_direction`, etc): Impacts are subtle, but red/blue coloring shows some directional effect (e.g., low pressure or strong wind may add small delay).
- `time_of_day_morning`: Morning time (red) increases predicted delay, as expected.
- `vehicle_bearing`, `vehicle_speed`: Higher bearing (towards downtown) and speed increases delay.
- `occupancy_status`: More passengers (red) increases delay.
- `stop_has_alert`: A stop being moved or cancelled (red) increases delay.
- `day_of_week`, `vehicle_in_transit`, `route_type_Day`: Mostly clustered near zero, which indicates low or inconsistent predictive power.

In [ ]:
# Force Plot (local explanation for one prediction)
sample_idx = 0
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[sample_idx], X_val_sample.iloc[sample_idx], matplotlib=True, show=False)

plt.title('SHAP Force Plot (local explanation for one prediction)', fontsize=18)
plt.tight_layout()
plt.savefig('../images/shap_force_plot_rf_reg_tuned.png', bbox_inches='tight')
plt.show()

**Positive Contributors (red)**

- relative_humidity = 44.0 → Strong positive influence. High humidity often correlates with slower transit times.
- vehicle_in_transit = 1.0 → The fact that the vehicle is currently in transit increases the prediction.
- vehicle_bearing = 0.0 → This might be linked to its orientation on the route, possibly indicating congestion or a stop.
- pressure = 1021.3 → This is somewhat surprising unless high pressure correlates with specific weather events.
- stop_lon = -73.6378 → Longitude might correlate with specific regions that experience more delays.
- stop_distance = 213.58 → Higher distance could imply more exposure to potential delays.

**Negative Contributors (blue)**

- trip_phase_end = 0 → This implies that the vehicle is still in the middle of its journey, reducing the delay expectation.
- occupancy_status = 1.0 → If the bus is at capacity, it might be on schedule.
- peak_hour = 0 → It's not peak hour, so less traffic is expected.
- day_of_week = 0 → Likely indicating it's a Sunday or Monday, which may have lighter traffic.

#### Feature Reduction Elimination Based on SHAP Plots

The goal is to remove one-hot features that have low average SHAP values, are clustered near zero in the beeswarm plot or don't show clear directional patterns.

In [ ]:
# Calculate mean SHAP per feature
shap_abs_mean = np.abs(shap_values).mean(axis=0)
shap_df = pd.DataFrame({
    'feature': X_val_sample.columns,
    'mean_abs_shap': shap_abs_mean
}).sort_values('mean_abs_shap', ascending=False)
shap_df

In [ ]:
# Display features with low SHAP score
threshold = 1.0
ohe_candidates = [col for col in X_test.columns if '_' in col and df[col].nunique() <= 2]
low_shap_ohe = shap_df[shap_df['feature'].isin(ohe_candidates) & (shap_df['mean_abs_shap'] < threshold)]
print('One-hot features with low SHAP impact to remove:')
print(low_shap_ohe)

In [ ]:
best_features = list(set(feature_cols).difference(set(low_shap_ohe['feature'].to_list())))
best_features

In [ ]:
# Keep best features
X_train_best = X_train[best_features]
X_train_sample_best = X_train_sample[best_features]
X_val_best = X_val[best_features]
X_test_best = X_test[best_features]

In [ ]:
# Retrain model on reduced features
rf_reg_retrained = RandomForestRegressor(
  	n_estimators=best_params['n_estimators'],
  	min_samples_split=best_params['min_samples_split'],
  	min_samples_leaf=best_params['min_samples_leaf'],
  	max_leaf_nodes=best_params['max_leaf_nodes'],
  	max_depth=best_params['max_depth'],
  	max_features=best_params['max_features'],
    bootstrap=best_params['bootstrap'],
  	random_state=42,
    n_jobs=-1
)

rf_reg_retrained.fit(X_train_sample_best, y_reg_train_sample)

In [ ]:
# Evaluate model
y_pred = rf_reg_retrained.predict(X_val_best)

metrics_df = add_metrics(metrics_df, rf_reg_retrained, y_pred, y_reg_val, 'rf_reg_retrained')
metrics_df[['model', 'MAE', 'RMSE', 'R²']]

#### Retune Parameters

In [ ]:
# Randomized search
random_search_rf = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    verbose=2,
    scoring=scoring,
    refit='RMSE',
    n_jobs=-1,
    random_state=42
)

random_search_rf.fit(X_train_sample_best, y_reg_train_sample)

In [ ]:
# Best model
rf_reg_final = random_search_rf.best_estimator_
best_params = random_search_rf.best_params_

#### Revalidate Model

In [ ]:
# Evaluate model
y_pred = rf_reg_final.predict(X_val_best)

metrics_df = add_metrics(metrics_df, rf_reg_final, y_pred, y_reg_val, 'rf_reg_final')
metrics_df[['model', 'MAE', 'RMSE', 'R²']]

In [ ]:
plot_residuals(y_pred, y_reg_val, 'rf_reg_final')

The metrics and the residual plots didn't change much after eliminating features. The R-squared is still very low, which indicates the Random Forest Regressor was not a good model choice for this dataset.

In [ ]:
# Save best parameters and features
rf_reg_attributes = {
	'best_params': best_params,
  	'best_features': best_features,
}

with open('../models/rf_reg_attributes.pkl', 'wb') as handle:
	pickle.dump(rf_reg_attributes, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Save model
joblib.dump(rf_reg_final, '../models/rf_reg_model.pkl')

### XGBoost Regressor

#### Fit Base Model

In [14]:
# Initialize the XGBoost model with default parameters
xg_reg_base = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

In [15]:
# Train a model
xg_reg_base.fit(X_train_sample, y_reg_train_sample)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

#### Evaluate model

In [16]:
y_pred = xg_reg_base.predict(X_val)

metrics_df = add_metrics(metrics_df, xg_reg_base, y_pred, y_reg_val, 'xg_reg_base')
metrics_df[['model', 'MAE', 'RMSE', 'R²']]

,model,MAE,RMSE,R²
0,rf_reg_base,107.140325,201.449965,-0.069803
1,xg_reg_base,97.216922,190.772235,0.040600


In [17]:
metrics_df.to_csv('metrics.csv', index=False)


**MAE:**

This is a very slight improvement over the previous models. The model is now, on average, 65.8 seconds off in its predictions, which is a reduction from the previous 69.6 seconds with the Random Forest and Gradient Boosting models.

**RMSE:**

The RMSE has also decreased compared to the previous ensemble model (117.83), indicating that the XGBoost model is performing better and has reduced the impact of large errors.

**R²:**

This is a substantial improvement from 0.247 (with the ensemble model). With an R-squared of 0,318%, the XGBoost model explains more variance in the data, which shows that it's capturing more of the underlying patterns than the previous models.

**Interpretation**

XGBoost is performing better than the previous models (Random Forest and Gradient Boosting ensemble), particularly in terms of MAE and RMSE. The higher R-squared value also suggests that the model is now explaining more of the variance in the target variable.

In [ ]:
# Plot residual analysis
plot_residuals(y_pred, y_reg_val, 'xg_reg_base')

The plots are very similar as the Random Forest ones.

#### Hyperparameter tuning

In [ ]:
# Create a GridSearchCV object
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(
				estimator=XGBRegressor(objective='reg:squarederror', random_state=42),
				param_grid=param_grid,
				cv=3,
				verbose=2,
				n_jobs=-1)

# Fit the model
grid_search.fit(X_train_sample, y_reg_train_sample)

In [ ]:
# Use the best model
xg_reg_tuned = grid_search.best_estimator_
best_params = grid_search.best_params_

In [ ]:
# Evaluate the best model
y_pred = xg_reg_tuned.predict(X_val)

metrics_df = add_metrics(metrics_df, xg_reg_tuned, y_pred, y_reg_val, 'xg_reg_tuned')
metrics_df[['model', 'MAE', 'RMSE', 'R²']]

**MAE:**

This is a small improvement over the previous MAE of 65.77 seconds. The model is now, on average, 64.2 seconds off in its predictions, which is a good step forward.

**RMSE:**

The RMSE has decreased again, from 112.11 to 110.89 seconds, indicating that the model has become slightly more accurate, especially in handling larger errors.

**R²:**

The R-squared value has increased marginally from 0.318 to 0.322, suggesting that the model is now explaining a bit more of the variance in the data, which is an improvement.

#### Feature Importances

In [ ]:
xg_reg_tuned = xg_reg_base #TODO: remove this

plot_importance(xg_reg_tuned, max_num_features=20)

plt.tight_layout()
plt.savefig('../images/feat_importance_xg_reg_runed.png', bbox_inches='tight')
plt.show()

#### SHAP Summary Plots

In [ ]:
# Create explainer with smaller set
X_val_sample = X_val[:1000]
explainer = shap.Explainer(xg_reg_tuned, X_val_sample)
shap_values = explainer(X_val_sample)

In [ ]:
# Summary plot (bar chart)
shap.summary_plot(shap_values, X_val_sample, plot_type='bar', show=False)

plt.title('SHAP Bar Plot (Global Impact)', fontsize=18)
plt.tight_layout()
plt.savefig('../images/shap_barplot_xg_reg_tuned.png', bbox_inches='tight')
plt.show()

- trip_phase_end - This feature is the most impactful. It likely represents the completion or nearing the end of the trip, significantly affecting delays.
- stop_lon and stop_lat - Geographic location of stops has a considerable impact, which may be related to traffic patterns or specific areas prone to delays.
- is_peak_hour - Being in peak hours naturally influences transit delays, as more congestion is expected.
- day_of_week - Certain days might have more predictable traffic or weather conditions affecting transit.
- stop_distance - The distance to the next stop could impact the time needed to reach it.

In [ ]:
# Summary plot (beeswarm: shows direction and spread)
shap.summary_plot(shap_values, X_val_sample, show=False)

plt.title('SHAP Beeswarm Plot (Local + Global Insight)', fontsize=18)
plt.tight_layout()
plt.savefig('../images/shap_beeswarm_plot_xg_reg_tuned.png', bbox_inches='tight')
plt.show()

trip_phase_end

Higher values push predictions to the right (increase in delay), while lower values reduce delay.

It appears when the bus is in its final phase of the route, it tends to experience more variability, potentially linked to traffic congestion near terminals or slower navigation in dense areas.

stop_lon and stop_lat

The spread in SHAP values indicates strong geographic influence.

Certain longitude and latitude combinations are clearly linked to either delays or smooth transit, suggesting geographical "hotspots."

is_peak_hour

Being a peak hour (high value = red) pushes the delay higher.

Non-peak hours (blue) tend to reduce delay predictions, aligning with traffic flow expectations.

day_of_week

There is a noticeable difference between weekdays and weekends.

Specific days (possibly Mondays and Fridays) might have more congestion or disruptions.

stop_distance

Greater distances between stops (red) tend to push delays up, potentially because of longer travel times.

Shorter distances (blue) slightly reduce delays.

pressure and temperature

These are quite dispersed, indicating that atmospheric pressure and temperature fluctuations can have a variable impact on travel time—likely weather-related.

occupancy_status

Higher occupancy (red) seems to slightly increase delays, possibly due to longer boarding times.

Key Takeaways:

Geographic Coordinates (stop_lon and stop_lat) are critical in predicting delays. This could suggest congestion is region-specific.

Trip Phase (trip_phase_end) is the most dominant feature—possibly because nearing the end of a trip increases variability in travel time.

Peak Hours and Day of Week are critical temporal indicators of delays, consistent with real-world traffic patterns.

Weather Attributes like pressure and temperature are influential but scattered, hinting at certain weather events causing spikes in delays.

In [ ]:
# Force Plot (local explanation for one prediction)
instance = X_val_sample.iloc[0:1]
shap_values = explainer(instance)

shap.initjs()
shap.force_plot(shap_values.base_values, shap_values.values, instance, matplotlib=True, show=False)

plt.title('SHAP Force Plot (local explanation for one prediction)', fontsize=18)
plt.tight_layout()
plt.savefig('../images/shap_force_plot_xg_reg_tuned.png', bbox_inches='tight')
plt.show()

**Positive Contributors (red)**

- relative_humidity = 44.0 → Strong positive influence. High humidity often correlates with slower transit times.
- vehicle_in_transit = 1.0 → The fact that the vehicle is currently in transit increases the prediction.
- vehicle_bearing = 0.0 → This might be linked to its orientation on the route, possibly indicating congestion or a stop.
- pressure = 1021.3 → This is somewhat surprising unless high pressure correlates with specific weather events.
- stop_lon = -73.6378 → Longitude might correlate with specific regions that experience more delays.
- stop_distance = 213.58 → Higher distance could imply more exposure to potential delays.

**Negative Contributors (blue)**

- trip_phase_end = 0 → This implies that the vehicle is still in the middle of its journey, reducing the delay expectation.
- occupancy_status = 1.0 → If the bus is at capacity, it might be on schedule.
- peak_hour = 0 → It's not peak hour, so less traffic is expected.
- day_of_week = 0 → Likely indicating it's a Sunday or Monday, which may have lighter traffic.

The model is heavily influenced by:

relative humidity and vehicle_in_transit for increasing delays.

trip_phase_end and occupancy_status for reducing the expected delay.

relative_humidity is the strongest factor in pushing the delay higher. This is likely because weather conditions tend to have a substantial impact on transit times.

The stop distance and longitude values indicate some spatial influence—perhaps certain regions are more prone to delays.

## Classification Model

## Final Model

### Evaluation

### Make Prediction

## End